In [5]:
import requests
import json
from pprint import pprint
from RiotRequest import RiotRequest

api_key = "RGAPI-889ae58e-f777-433a-ab22-dbcb3cda10d2"

def get_account_id(summoner_name):
    '''
    input: 접근할 수 있는 summoner name
    output: account_id
    '''
    json_data = RiotRequest("https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/{0}?api_key={1}".format(summoner_name, api_key))
    return json_data['accountId']


def get_match_ids(account_id):
    '''
    matches라는 key 안에 최대 최근 100개의 match id만
    API doc: https://developer.riotgames.com/api-methods/#match-v4/GET_getMatchlist
    '''
    json_data = RiotRequest("https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/{0}?api_key={1}".format(account_id, api_key))
    game_id_list = [match['gameId'] for match in json_data['matches']]
    return game_id_list


def get_history(account_id):
    beginIndex = 0
    queue = 'RANKED_SOLO_5x5'
    json_data = RiotRequest("https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/{0}?api_key={1}&beginIndex={2}&queue=RANKED_SOLO_5x5".format(account_id, api_key, beginIndex))
    
    return json_data
    
def get_match_history(account_id):
    '''
    account_id: str. accound id of the user
    output: List[MatchReferenceDto]
    MatchReferenceDto: {'platformId', 'gameId', 'champion', 'queue', 'season', 'timestamp', 'role', 'lane'}
    API doc: https://developer.riotgames.com/api-methods/#match-v4/GET_getMatchlist
    '''   
    # initialize
    beginIndex = 0
    matchList = []
    
    # make multiple requests to get all matches
    while True: 
        url = "https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/{0}?api_key={1}&beginIndex={2}".format(account_id, api_key, beginIndex)
        json_data = RiotRequest(url)
        
        # save match data into list
        matchList.extend([match['gameId'] for match in json_data['matches']])

        if json_data['endIndex'] >= json_data['totalGames']: # if endIndex reaches the number of total games
            break # terminate the while loop
        else: # make a request on previous 100 matches
            beginIndex += 100
    
    return matchList

def get_match_history_by_season(account_id, season):
    # beginTime/endTime of each season: https://rankedboost.com/season-end-league-of-legends/
    pass


def get_game_detail(game_id):
    json_data = RiotRequest("https://kr.api.riotgames.com/lol/match/v4/matches/{0}?api_key={1}".format(game_id, api_key))
    return json_data

def get_match_timeline(game_id):
    '''
    1분 단위로 나뉘어진 프레임의 리스트를 반환
    List[{'timestamp': int, 'participantFrames': Map[String, MatchParticipantFrameDto], 'events': List[MatchEventDto]}]
    '''
    json_data = RiotRequest("https://kr.api.riotgames.com/lol/match/v4/timelines/by-match/{0}?api_key={1}".format(game_id, api_key))
    return json_data['frames']

In [2]:
#name of CHALLENGER
queue = 'RANKED_SOLO_5x5'
json_data = RiotRequest("https://kr.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/{0}?api_key={1}".format(queue, api_key))
CHALLENGER = [i['summonerId'] for i in json_data['entries']]


ACCOUNT_ID 구하기 

In [3]:
def account_id(summonerId):
    '''
    input: 접근할 수 있는 summoner Id
    output: account_id
    '''
    try :
        json_data = RiotRequest("https://kr.api.riotgames.com/lol/summoner/v4/summoners/{0}?api_key={1}".format(summonerId, api_key))
    except: TypeError
    
    return json_data['accountId']

In [4]:
#ACCOUNT_ID 구하기
account_CH = []
for i in CHALLENGER:
    account_CH.append(account_id(i))

Exceeded Rate Limiting. Will retry After 68 seconds
Exceeded Rate Limiting. Will retry After 78 seconds
Exceeded Rate Limiting. Will retry After 79 seconds


MATCH_ID 구하기

In [36]:
#match_id 구하기


match_CH = []
for i in account_CH:
    match_CH.append(get_match_ids(i))

Unhandled Error 400 : https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/dZtosebu4TR2TXEG-aYN80twDz3uZIR52gq2BeJuHp_WZ8dzA2V-wUmf?api_key=RGAPI-889ae58e-f777-433a-ab22-dbcb3cda10d2&queue=RANKED_SOLO_5x5


TypeError: 'NoneType' object is not subscriptable

In [1]:
#피클파일
import pickle
with open("account_CH.pickle", 'rb') as f:
    account_CH = pickle.load(f)
with open("match_CH.pickle", "rb") as f:
    match_CH = pickle.load(f)
with open("match_CH_dict.pickle", "rb") as f:
    match_CH_dict = pickle.load(f)

In [ ]:
#match와 account_id를 이용해 딕셔너리 만들기 account_id가 키
match_CH_dict = {}
i=0
for id in account_CH:
    match_CH_dict[id]= match_CH[i]
    i+=1

In [5]:
win_rate_all = {}
error_list = {}

for account in match_CH_dict: #match_CH_dict에 저장된 account_id를 돌림
    print(account)
    win = 0  # 어짜피 100경기를 돌리기 때문에 이긴 경기를 전부 더하면 승률이 된다
    for game in range(0,len(match_CH_dict[account])): #저장된 게임만큼 여기서는 100경기를 돌린다,
        game_detail = get_game_detail(match_CH_dict[account][game]) #시간을 단축하기 위해 미리 게임 디테일을 뽑아 놓는다
        tr #이상한 503 에러때문에 try except 사용 
            for i in range(0,len(game_detail['participantIdentities'])):  #한 경기에 10명의 플레이어만 있다고 생각했으나, 아닌 것도 있어서 이렇게 플레이어 전체를 돌림        
                if game_detail['participantIdentities'][i]['player']['accountId'] ==  account: #10명의 플레이어중 지금 우리가 조사하는 사람의 승을 알고 싶기 때문에, 원래의 키값과 같은 사람을 골라낸다.  
                    if game_detail['participants'][i]['stats']['win'] == True: #만약 그 사람이 이겼는지 졌는지 를 확인하고
                        win+=1 #이기면 +1 
        except: #이건 오류난거 저장
            error_game_number = match_CH_dict[account][game]
            print("error occurred for game {0}".format(error_game_number))
            error_list[account] = error_game_number
            continue
    print(win)
    win_rate_all[account] = win

dZtosebu4TR2TXEG-aYN80twDz3uZIR52gq2BeJuHp_WZ8dzA2V-wUmf
56
BkKH6CqwnD0OiCUj8KMCEKUPvT16BZzxCr0BdW-GA0FV
Exceeded Rate Limiting. Will retry After 83 seconds
52
qRGmB4ny5HUUHS3s7zvKfMn7iYuKCjJrFLz-Ws7CrqmXrAwpX19th9jk
Exceeded Rate Limiting. Will retry After 83 seconds
56
guwRATYQC78fRNS_fnztoprjtwhuXDDH4YwK4L-I_66XYVk
Exceeded Rate Limiting. Will retry After 83 seconds
53
B494lJrFQkxnp73DFuIMv0V2RiIvjW0rN8Z0bJS1WMY19U8
Exceeded Rate Limiting. Will retry After 85 seconds
58
t0an09MAcCMv_3zwG9k43WbQldmaVmRf0KYBUgcs8CsrAJKJYoCHe0pr
Exceeded Rate Limiting. Will retry After 83 seconds
48
gA-_lp9fLS5vZhxrbtFeIHT9CY_3pzHtH2JHk8QdS8b91g0
Exceeded Rate Limiting. Will retry After 83 seconds
53
mg5dggWtAJVGxuiHcVJ9NJv5jLJbtRo7zrxx2Fk6OglT
Exceeded Rate Limiting. Will retry After 84 seconds
48
2m7ynGVKiMT97ysOQOgQYLloFTAMaHdpqZ7vpiznOTO9fM0
Exceeded Rate Limiting. Will retry After 85 seconds
60
Hs3CkqjWhCcrS105X-d-zeX-ismN_mWBYNDipI4jkg8uIzg
Exceeded Rate Limiting. Will retry After 84 seconds
56
P

45
djttXUEgsufK29ufIt_XrNzpi0BO0ufctaLn16jHm5cC9yYuoL4YGRnO
Exceeded Rate Limiting. Will retry After 84 seconds
57
d5j11c2EggvyjLps61vq3gF0FkuEDTHVdpiGiFXAqZJT35PvBONs6ns0
Exceeded Rate Limiting. Will retry After 83 seconds
56
ATeoaYPp8dn02JwslZs3TCjPzHYaGLl_9dAibHFEkdURKy88pg3NmMSh
Exceeded Rate Limiting. Will retry After 84 seconds
57
USt0VWvzNit9DKo2QHgOpJOeqr1Gzi2cOAVI0QzmcX5oYRUo8h-h_R4n
Exceeded Rate Limiting. Will retry After 84 seconds
54
9kw5U_VRwdgGs56h5r7VBK-iDE200M_v9J2f28bduJZiJ8w
Exceeded Rate Limiting. Will retry After 84 seconds
50
zCsfkZQgr8WKLAEyFbX2WlNM35XBqiQRP8BUYMwxdJSSsfg
Exceeded Rate Limiting. Will retry After 82 seconds
51
WqhPqFx4TnFemBa71OpWyb8pKQe-n7aX6KAJkhq9Y9yUy48
Exceeded Rate Limiting. Will retry After 84 seconds
54
6GpvD_UlMB9LGC5ru4Jr3XSq7rxREYn7ih8By3w-lspni78
Exceeded Rate Limiting. Will retry After 84 seconds
52
ZcOXlxJzBMgB5xNQIoULi8oGH4_SrdU_gupvYnMPGvg81cU
Exceeded Rate Limiting. Will retry After 84 seconds
63
wOZvKjQ1L0J8LOvfK5OOAmiTSnYpPQvomt

46
4eBrfJwNyCYBpSwTMdNZxwqTJFqRqGyvAo-tPKsM3N6J6rPOuemvF3je
Exceeded Rate Limiting. Will retry After 81 seconds
50
D32ppxSlppfQhX_6pVx9q4_9fyLg3c4Baje9O2POycwQJHJfE23Ha39_
Exceeded Rate Limiting. Will retry After 81 seconds
55
Q67u-2n04KdHe-hYcl8__q6SThrmpGoDBF0vsIWgcdFcuDdHIuYbDc7R
Exceeded Rate Limiting. Will retry After 81 seconds
52
F5Nw3LVqLPwMWpoRRYq-LUhmHsURhSJMZjSwHSBEgIRGUjI
Exceeded Rate Limiting. Will retry After 79 seconds
71
j2c-Oa7NB4_pE5SwEcj2r12WRcylg8GtgD7kQ6vyo5WG
Exceeded Rate Limiting. Will retry After 82 seconds
48
zZsQWaOShPiAL40i6X-_db1wL3IS57-M47puZoALczBny08
Exceeded Rate Limiting. Will retry After 80 seconds
58
xELsdf6ugVwRfzAVTKO2BQRbbQk9D3SzX0-7gQzjBEZCYds
Exceeded Rate Limiting. Will retry After 80 seconds
62
idmzHLZzbf9ldpS_Kan8nIGnMnoE_bh6MDl2Qyt33T2TIFM
Exceeded Rate Limiting. Will retry After 82 seconds
54
7t49L7KrVczc1uUK562hHDCgZjxXqeNhpGgqYF6f33Edi54
Exceeded Rate Limiting. Will retry After 82 seconds
53
FJeox_vcFur54-F5r7qmt3vQUeak-Xy7RygKFM4tHHS6JZ

56
eXLn7hwj8pizYJoQ95ftuJe_mdaZg3Y3N-UwRgQCLzYKzC4
Exceeded Rate Limiting. Will retry After 82 seconds
51
2z9dDvFG6Z9GBENtM1GOj_a8RlrlY7PEbsoXfwDJSzYx
Exceeded Rate Limiting. Will retry After 81 seconds
46
nc7iLpecgyejksvR0X0xzZo7d5KoPzwLb9iq7I9glshZHnI
Exceeded Rate Limiting. Will retry After 82 seconds
51
BU9G8GT1-MTC1UcQsIpEH6icZZF_R661pb_XvtI0FF_Y67qODHmqmv2t
Exceeded Rate Limiting. Will retry After 82 seconds
52
mYtXIKGIsnLoDYufyVJt5X17rA_3DAABKgpUrXAZ02BlFMRxql5dhS1g
Exceeded Rate Limiting. Will retry After 81 seconds
61
mTUsxpB-fVOcslaK2qNXnzckzpoHRcnus71eMb3fi0Uj6oU
Exceeded Rate Limiting. Will retry After 81 seconds
50
JcpMr8Nw4UNC95KlKpqHaJb02zJF_M6oOm_KnLQs6KxZjTM
Exceeded Rate Limiting. Will retry After 81 seconds
49
ZSWpJePXgMF-qqlC-UFJlrBLjx0EjOuZmutfm4pYj7upCIs
Exceeded Rate Limiting. Will retry After 82 seconds
62
bHyfiYZ-hQ48x3AU-XjJbdu5GLKkzByz4sMyEwp8sYNCyYO8BIN3rPCC
Exceeded Rate Limiting. Will retry After 81 seconds
49
K-Q7GwjTkNxE01wHqGusTud3CpGnQkkynOfBkBJaMpZ5nR

52
cEMAlVd7Yeds-uxGhOmAh0hDENoCS2o45YrrZ-CuhsCmRY9mByah_id4
Exceeded Rate Limiting. Will retry After 81 seconds
55
U8YdvBbRbdI2topCQfmWEI9W_plUh8JKyz4cRcDdHN3xHI3h8a_NI-IN
Exceeded Rate Limiting. Will retry After 81 seconds
60
h9-WJMVyvDyrGpU3-jEqrqdJpi96kdvfV3OSihG56UeQe30
Exceeded Rate Limiting. Will retry After 81 seconds
46
GL8PpgxYVSEbIAPeflYol3T-kNWxcqK9qcUJAE5nT2uUoOM
Exceeded Rate Limiting. Will retry After 80 seconds
58
LgVxriQfwkyXLU6sqV24j8BY4T0Eh2J-o4T3ftjbo1pX
Exceeded Rate Limiting. Will retry After 81 seconds
49
go-jdrQwRe3rdsGQyvu-dwnCtKaiaDxowrW2RGrEeV2K3Gg
Exceeded Rate Limiting. Will retry After 81 seconds
53
iC8Qf2sGt7ucnm0YwSG4AnFPzy5reWdWneQRQJWMttQ_vmYDuHOuyW4c
Exceeded Rate Limiting. Will retry After 81 seconds
58
FdfbKbQgyqWO8Ld9x7Sb9AvaNbAcfa6sbtKL3VNFqbWm-4M_HTWcyIRx
Exceeded Rate Limiting. Will retry After 81 seconds
54
GpNDyk7qfCdcE0c0_RslEjm8PudxTwQ3ODHH66-h8Jtd
Exceeded Rate Limiting. Will retry After 81 seconds
Unhandled Error 500 : https://kr.api.riotga

In [6]:
win_rate_all

{'dZtosebu4TR2TXEG-aYN80twDz3uZIR52gq2BeJuHp_WZ8dzA2V-wUmf': 56,
 'BkKH6CqwnD0OiCUj8KMCEKUPvT16BZzxCr0BdW-GA0FV': 52,
 'qRGmB4ny5HUUHS3s7zvKfMn7iYuKCjJrFLz-Ws7CrqmXrAwpX19th9jk': 56,
 'guwRATYQC78fRNS_fnztoprjtwhuXDDH4YwK4L-I_66XYVk': 53,
 'B494lJrFQkxnp73DFuIMv0V2RiIvjW0rN8Z0bJS1WMY19U8': 58,
 't0an09MAcCMv_3zwG9k43WbQldmaVmRf0KYBUgcs8CsrAJKJYoCHe0pr': 48,
 'gA-_lp9fLS5vZhxrbtFeIHT9CY_3pzHtH2JHk8QdS8b91g0': 53,
 'mg5dggWtAJVGxuiHcVJ9NJv5jLJbtRo7zrxx2Fk6OglT': 48,
 '2m7ynGVKiMT97ysOQOgQYLloFTAMaHdpqZ7vpiznOTO9fM0': 60,
 'Hs3CkqjWhCcrS105X-d-zeX-ismN_mWBYNDipI4jkg8uIzg': 56,
 'PA8keq17Il74Ywi58ft1IijSOted9oDEbfMWZkgazh5JKUhuJh3Klkfx': 46,
 'xWH_Bywryg57X4-9g62OSJ2Si8xXkzOL0CalfTsfV8o3vRE': 66,
 'IOQjr22mIGRl4QPIdsPCAmlkqL00t3bZR7vLHHuABQ0Gqh4': 49,
 '_2eXGB8DpDoeKRioQk5MeqW67jIEFgauH8vkLF1RpEQymldWh9DiSAlk': 49,
 'Q-4h4A2Y_x4HjXnTdn1gnBTh042gDnC6lAxIgeV8ZpAlYQN3AXV28gU7': 56,
 'VdfdQ796I3T_EYGUD0_19_8QlChkAxPd2fsM3OxXn3ZRtSY': 56,
 'KgOHwTcke60_t9UjRwHngYA-wWGSkMxAh05Gqq8GmVE': 56,
 'Th

In [7]:
with open("win_rate_all.pickle", "wb") as fw:
    pickle.dump(win_rate_all, fw)